In [1]:
from ultralytics.data.converter import convert_coco

In [3]:
convert_coco (
    labels_dir='data',
    save_dir='yolo_annos',
    use_keypoints= False,
    use_segments=True
)

Annotations /Users/mitchellpalmer/Projects/solafune-canopy-capstone-clean/data/train.json: 100%|██████████| 150/150 [00:00<00:00, 281.76it/s]

COCO data converted successfully.
Results saved to /Users/mitchellpalmer/Projects/solafune-canopy-capstone-clean/yolo_annos2


2. Automatic dataset split with Python

If all your images are currently in one folder (with labels alongside), you can split them like this:

In [4]:
import os, random, shutil
from pathlib import Path

# Paths
IMG_DIR = Path("model-data/images copy/train")
LBL_DIR = Path("model-data/labels copy/train")

OUT_DIR = Path("data")
splits = {"train": 0.7, "val": 0.3, "test": 0.0}  # 70/30/00 split

# Collect all images
images = list(IMG_DIR.glob("*.jpg")) + list(IMG_DIR.glob("*.png")) + list(IMG_DIR.glob("*.tif")) # glob is from package Path
random.shuffle(images)

# Split indices
n = len(images)
train_end = int(splits["train"] * n)
val_end = train_end + int(splits["val"] * n)

datasets = {
    "train": images[:train_end],
    "val": images[train_end:val_end],
    "test": images[val_end:],
}

# Copy files into YOLO structure
for split, files in datasets.items():
    (OUT_DIR / "images" / split).mkdir(parents=True, exist_ok=True)
    (OUT_DIR / "labels" / split).mkdir(parents=True, exist_ok=True)

    for img in files:
        label = LBL_DIR / (img.stem + ".txt")
        shutil.copy(img, OUT_DIR / "images" / split / img.name)
        if label.exists():
            shutil.copy(label, OUT_DIR / "labels" / split / label.name)


In [7]:
import torch
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

True
True


In [8]:
import torch
torch.mps.empty_cache()

In [ ]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="coco8.yaml", epochs=3)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg")

# Export the model to ONNX format
success = model.export(format="onnx")